In [122]:
import pandas as pd
file_path = "/Users/vee/Desktop/nasdaq-enrichment/"

nasdaq_df = pd.read_csv(file_path + "nasdaq100.csv")
price_change_df = pd.read_csv(file_path + "nasdaq100_price_change.csv")

print("Company Information:")
display(nasdaq_df.head())

print("\nStock Price Changes:")
display(price_change_df.head())



Company Information:


,symbol,name,headQuarter,dateFirstAdded,cik,founded
0,AAPL,Apple Inc.,"Cupertino, CA",NaN,320193,1976-04-01
1,ABNB,Airbnb,"San Francisco, CA",NaN,1559720,2008-08-01
2,ADBE,Adobe Inc.,"San Jose, CA",NaN,796343,1982-12-01
3,ADI,Analog Devices,"Wilmington, MA",NaN,6281,1965-01-01
4,ADP,ADP,"Roseland, NJ",NaN,8670,1949-01-01



Stock Price Changes:


,symbol,1D,5D,1M,3M,6M,ytd,1Y,3Y,5Y,10Y,max
0,AAPL,-1.7254,-8.30086,-6.20411,3.04200,15.64824,42.99992,8.479410,60.96299,245.42031,976.99441,139245.53954
1,ABNB,2.1617,-2.21919,9.88336,19.43286,19.64241,68.66902,23.640130,-1.04347,-1.04347,-1.04347,-1.04347
2,ADBE,0.5409,-1.77817,9.16191,52.04650,38.01522,57.22723,21.962060,17.83037,109.05718,1024.69214,251030.66399
3,ADI,0.9291,-4.03352,2.58486,3.65887,5.01602,17.02062,8.097350,63.42847,92.81874,286.77518,26012.63736
4,ADP,2.0589,2.35462,14.66581,16.40059,10.60546,5.53732,0.888943,81.76679,81.87224,248.40950,27613.11042


In [2]:
print("Missing values in 'nasdaq100.csv':")
print(nasdaq_df.isnull().sum())

print("\nMissing values in 'nasdaq100_price_change.csv':")
print(price_change_df.isnull().sum())


Missing values in 'nasdaq100.csv':
symbol              0
name                0
headQuarter         0
dateFirstAdded    101
cik                 0
founded             0
dtype: int64

Missing values in 'nasdaq100_price_change.csv':
symbol    0
1D        0
5D        0
1M        0
3M        0
6M        0
ytd       0
1Y        0
3Y        0
5Y        0
10Y       0
max       0
dtype: int64


In [3]:
# Check for duplicates in both datasets
print("\nDuplicate rows in 'nasdaq100.csv':")
print(nasdaq_df.duplicated().sum())

print("\nDuplicate rows in 'nasdaq100_price_change.csv':")
print(price_change_df.duplicated().sum())

# Check for duplicate symbol
print("\nDuplicate symbols in 'nasdaq100.csv':")
print(nasdaq_df['symbol'].duplicated().sum())

print("\nDuplicate symbols in 'nasdaq100_price_change.csv':")
print(price_change_df['symbol'].duplicated().sum())



Duplicate rows in 'nasdaq100.csv':
0

Duplicate rows in 'nasdaq100_price_change.csv':
0

Duplicate symbols in 'nasdaq100.csv':
0

Duplicate symbols in 'nasdaq100_price_change.csv':
0


In [4]:
# Check the number of rows in each dataframe
print("Rows in nasdaq100.csv:", nasdaq_df.shape[0])
print("Rows in nasdaq100_price_change.csv:", price_change_df.shape[0])


Rows in nasdaq100.csv: 101
Rows in nasdaq100_price_change.csv: 101


In [7]:
# Check the data type of 'dateFirstAdded'
print(nasdaq_df['dateFirstAdded'].dtype)


float64


In [ ]:
# Convert 'dateFirstAdded' to datetime type
nasdaq_df['dateFirstAdded'] = pd.to_datetime(nasdaq_df['dateFirstAdded'], errors='coerce')

# Check if mode has a valid value
if not nasdaq_df['dateFirstAdded'].mode().empty:
    most_frequent_date = nasdaq_df['dateFirstAdded'].mode()[0]
    nasdaq_df['dateFirstAdded'] = nasdaq_df['dateFirstAdded'].fillna(most_frequent_date)
else:
    # If mode is empty, fill with a placeholder date 
    nasdaq_df['dateFirstAdded'] = nasdaq_df['dateFirstAdded'].fillna(pd.to_datetime('1900-01-01'))
print(nasdaq_df['dateFirstAdded'].isnull().sum())


0


In [10]:
nasdaq_df

,symbol,name,headQuarter,dateFirstAdded,cik,founded
0,AAPL,Apple Inc.,"Cupertino, CA",1900-01-01,320193,1976-04-01
1,ABNB,Airbnb,"San Francisco, CA",1900-01-01,1559720,2008-08-01
2,ADBE,Adobe Inc.,"San Jose, CA",1900-01-01,796343,1982-12-01
3,ADI,Analog Devices,"Wilmington, MA",1900-01-01,6281,1965-01-01
4,ADP,ADP,"Roseland, NJ",1900-01-01,8670,1949-01-01
...,...,...,...,...,...,...
96,WBD,Warner Bros. Discovery,"New York, NY",1900-01-01,1437107,2022-04-08
97,WDAY,"Workday, Inc.","Pleasanton, CA",1900-01-01,1327811,2005-03-01
98,XEL,Xcel Energy,"Minneapolis, MN",1900-01-01,72903,1909-06-17
99,ZM,Zoom Video Communications,"San Jose, CA",1900-01-01,1585521,2011-04-21


In [12]:
from dotenv import load_dotenv
import os

load_dotenv()  

openai_key = os.getenv("OPENAI_API_KEY")
print("Key loaded successfully!")

Key loaded successfully!


### Classifying Companies into Sectors

In [ ]:
import openai
import os
import pandas as pd
from dotenv import load_dotenv
from tqdm import tqdm
from functools import lru_cache

load_dotenv()
client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Classification function
@lru_cache(maxsize=100)
def classify_sector(company_name):
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{
                "role": "user", 
                "content": f"Classify {company_name} into one business sector (respond with only the sector name)"
            }],
            max_tokens=10,
            temperature=0.2
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error classifying {company_name}: {e}")
        return None
nasdaq_df['sector'] = [classify_sector(name) for name in tqdm(nasdaq_df['name'])]

print(nasdaq_df[['symbol', 'name', 'sector']].head())
nasdaq_df.to_csv("nasdaq100_with_sectors.csv", index=False)

100%|██████████| 101/101 [00:50<00:00,  2.02it/s]

  symbol            name                  sector
0   AAPL      Apple Inc.              Technology
1   ABNB          Airbnb             Hospitality
2   ADBE      Adobe Inc.              Technology
3    ADI  Analog Devices  Semiconductor Industry
4    ADP             ADP       Business Services


In [ ]:
nasdaq_df

In [43]:
print(nasdaq_df['sector'].unique())


['Technology' 'Hospitality' 'Semiconductor Industry' 'Business Services'
 'Utilities' 'Healthcare' 'Biotechnology' 'E-commerce'
 'Engineering Software' 'Travel and Tourism' 'Energy Services'
 'Energy Sector' 'Telecommunications' 'Retail' 'Automotive Industry'
 'Cybersecurity' 'Information Technology' 'Real Estate Technology'
 'Transportation' 'Industrial Services' 'Information Technology (IT)'
 'Renewable Energy' 'Industrial Distribution'
 'Semiconductor Manufacturing' 'Industrial Conglomerate'
 'Healthcare Technology' 'Beverage sector' 'Consumer Packaged Goods'
 'Automotive industry' 'Semiconductors' 'Consumer Staples' 'Entertainment'
 'Transportation and Logistics' 'Retail Sector' 'Automotive' 'Real Estate'
 'Food and Beverage' 'Financial Technology (FinTech)'
 'Media & Entertainment' 'Financial Services']


####  inconsistencies in the way some sectors are labeled, such as:

"Technology" vs. "Information Technology"

"Automotive Industry" vs. "Automotive"

"Semiconductor Industry" vs. "Semiconductors"

"Retail" vs. "Retail Sector"

And others with variations like "Food and Beverage" vs. "Beverage sector

#### 40 sector classified by AI

### Mapping sector into the same categories

In [94]:
sector_mapping = {
    'Technology': 'Technology',
    'Information Technology': 'Technology',
    'Automotive Industry': 'Automotive',
    'Automotive': 'Automotive',
    'Semiconductor Industry': 'Semiconductors',
    'Semiconductors': 'Semiconductors',
    'Retail Sector': 'Retail',
    'Retail': 'Retail',
    'Healthcare Technology': 'Healthcare',
    'E-commerce': 'E-commerce',
    'Cybersecurity': 'Cybersecurity',
    'Consumer Packaged Goods': 'Consumer Goods',
    'Media & Entertainment': 'Entertainment',
    'Financial Technology (FinTech)': 'FinTech',
    'Energy Sector': 'Energy',
    'Food and Beverage': 'Food and Beverage',
    'Beverage sector': 'Food and Beverage',
    'Industrial Services': 'Industrial Services',
    'Transportation and Logistics': 'Transportation',
    'Transportation': 'Transportation',
    'Engineering Software': 'Software',
    'Industrial Distribution': 'Industrial Services',
    'Industrial Conglomerate': 'Industrial Conglomerate',
    'Real Estate Technology': 'Real Estate',
    'Real Estate': 'Real Estate',
    'Renewable Energy': 'Energy',
    'Semiconductor Manufacturing': 'Semiconductors',
    'Business Services': 'Business Services',
    'Telecommunications': 'Telecommunications',
    'Healthcare': 'Healthcare',
    'Utilities': 'Utilities',
    'Financial Services': 'Finance',
    'Consumer Staples': 'Consumer Goods',
    'Entertainment': 'Entertainment',
    'Automotive industry': 'Automotive',
    'Transportation': 'Transportation',
    'Automotive Industry': 'Automotive'
}
# Applying the mapping into sector column
nasdaq_df['sector'] = nasdaq_df['sector'].map(sector_mapping).fillna(nasdaq_df['sector'])
print(nasdaq_df['sector'].unique())

['Technology' 'Hospitality' 'Semiconductors' 'Business Services'
 'Utilities' 'Healthcare' 'Biotechnology' 'E-commerce' 'Software'
 'Travel and Tourism' 'Energy Services' 'Energy' 'Telecommunications'
 'Retail' 'Automotive' 'Cybersecurity' 'Real Estate' 'Transportation'
 'Industrial Services' 'Information Technology (IT)'
 'Industrial Conglomerate' 'Food and Beverage' 'Consumer Goods'
 'Entertainment' 'FinTech' 'Finance']


/var/folders/qf/27x4m9xd7930x_n62xzf37kc0000gn/T/ipykernel_51708/2564901587.py:41: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [95]:
print(nasdaq_df['sector'].nunique())

26


In [ ]:
# Applying the mapping into the 'sector' column
nasdaq_df.loc[:, 'sector'] = nasdaq_df['sector'].apply(lambda x: sector_mapping.get(x, x))


###  Merging the Datasets

In [106]:
# Merge the two data on symbol
company_performance_summary = pd.merge(nasdaq_df, price_change_df, on='symbol', how='left')
print(company_performance_summary.head())


  symbol                     name        headQuarter dateFirstAdded      cik  \
0   AAPL               Apple Inc.      Cupertino, CA     1900-01-01   320193   
1   ABNB                   Airbnb  San Francisco, CA     1900-01-01  1559720   
2    ADI           Analog Devices     Wilmington, MA     1900-01-01     6281   
3    ADP                      ADP       Roseland, NJ     1900-01-01     8670   
4    AEP  American Electric Power       Columbus, OH     1900-01-01     4904   

      founded             sector      1D       5D        1M        3M  \
0  1976-04-01         Technology -1.7254 -8.30086  -6.20411   3.04200   
1  2008-08-01        Hospitality  2.1617 -2.21919   9.88336  19.43286   
2  1965-01-01     Semiconductors  0.9291 -4.03352   2.58486   3.65887   
3  1949-01-01  Business Services  2.0589  2.35462  14.66581  16.40059   
4  1906-01-01          Utilities -0.4431 -3.58803  -4.04556 -12.19194   

         6M       ytd         1Y        3Y         5Y        10Y           max  

## Sector Performance Summary

### Calculating Sector-wise Average Stock Performance

In [108]:
# Grouping by 'sector' and calculate the average performance for each time frame
sector_performance = company_performance_summary.groupby('sector')[['1D', '5D', '1M', '3M', '6M', 'ytd', '1Y', '3Y', '5Y', '10Y', 'max']].mean()

# Reset index to include 'sector' as a regular column for easy access
sector_performance = sector_performance.reset_index()
print(sector_performance)


                         sector      1D         5D         1M         3M  \
0                    Automotive  1.2790   1.589460   1.554870  13.423750   
1                 Biotechnology  3.9707   8.688550  15.682600   6.937810   
2             Business Services  2.0589   2.354620  14.665810  16.400590   
3                Consumer Goods -0.7936  -1.547120  -1.297240 -15.295260   
4                 Cybersecurity  0.3190  -6.527650   4.803890  23.291140   
5                    E-commerce  1.8987   9.270560   9.585450  34.607920   
6                        Energy -0.1828   8.084180  13.599040  30.519560   
7               Energy Services  0.7644   0.197072   7.490180  26.116230   
8                 Entertainment  2.1223   1.270590   0.607167  36.559670   
9                       FinTech  2.6614 -13.390700  -3.548440 -14.072300   
10                      Finance  1.6408   3.158410   3.804950  10.740710   
11            Food and Beverage  0.6548  -0.995316   6.486150   1.775500   
12          

In [123]:
import plotly.express as px

time_frame = '1Y'

sector_performance_subset = sector_performance[['sector', time_frame]]
sector_performance_subset = sector_performance_subset.sort_values(by=time_frame, ascending=False)

fig = px.bar(sector_performance_subset,
             x='sector', 
             y=time_frame,
             title=f"Average Sector Performance Over Time ({time_frame})",
             labels={"sector": "Sector", time_frame: "Performance (%)"},
             barmode='group')

fig.update_layout(xaxis_tickangle=-45)
fig.show()


### Preparing data for openAI

####  Extracing Top and Bottom Performing Sectors

In [124]:
# Top and bottom 3 performing sectors based on max
top_sectors = sector_performance.nlargest(3, '1Y')[['sector', '1Y']]
bottom_sectors = sector_performance.nsmallest(3, '1Y')[['sector', '1Y']]

# top and bottom sectors for API input
top_sectors_str = "\n".join([f"sector: {row['sector']}, {row['1Y']}" for _, row in top_sectors.iterrows()])
bottom_sectors_str = "\n".join([f"sector: {row['sector']}, {row['1Y']}" for _, row in bottom_sectors.iterrows()])

print("Top Performing Sectors:")
print(top_sectors_str)
print("\nBottom Performing Sectors:")
print(bottom_sectors_str)


Top Performing Sectors:
sector: Entertainment, 88.7704
sector: Travel and Tourism, 65.81501
sector: Energy Services, 46.64194

Bottom Performing Sectors:
sector: FinTech, -33.07708
sector: Cybersecurity, -20.34086
sector: Utilities, -19.18465


#### Top and Bottom Performing Companies

In [125]:
top_companies = company_performance_summary.nlargest(3, '1Y')[['name', '1Y']]
bottom_companies = company_performance_summary.nsmallest(3, '1Y')[['name', '1Y']]

# API input
top_companies_str = "\n".join([f"{row['name']} {row['1Y']}" for _, row in top_companies.iterrows()])
bottom_companies_str = "\n".join([f"{row['name']} {row['1Y']}" for _, row in bottom_companies.iterrows()])

print("\nTop Performing Companies:")
print(top_companies_str)
print("\nBottom Performing Companies:")
print(bottom_companies_str)


Top Performing Companies:
Netflix 88.7704
Booking Holdings 65.81501
Baker Hughes 46.64194

Bottom Performing Companies:
PayPal -33.07708
CrowdStrike -20.34086
American Electric Power -19.18465


#### Combine the Data and Prepare for API

In [126]:
# Combine all the data for API input
api_input = f"""
Top Performing Sectors:
{top_sectors_str}

Bottom Performing Sectors:
{bottom_sectors_str}

Top Performing Companies:
{top_companies_str}

Bottom Performing Companies:
{bottom_companies_str}
"""
print("\nAPI Input:\n")
print(api_input)



API Input:


Top Performing Sectors:
sector: Entertainment, 88.7704
sector: Travel and Tourism, 65.81501
sector: Energy Services, 46.64194

Bottom Performing Sectors:
sector: FinTech, -33.07708
sector: Cybersecurity, -20.34086
sector: Utilities, -19.18465

Top Performing Companies:
Netflix 88.7704
Booking Holdings 65.81501
Baker Hughes 46.64194

Bottom Performing Companies:
PayPal -33.07708
CrowdStrike -20.34086
American Electric Power -19.18465



### Generating Insights with OpenAI

In [127]:
def prepare_insights_for_openai():
    # Preparing the string input for top and bottom sectors
    top_sectors_str = "\n".join([f"Sector: {row['sector']}, Performance: {row['1Y']:.2f}" for _, row in top_sectors.iterrows()])
    bottom_sectors_str = "\n".join([f"Sector: {row['sector']}, Performance: {row['1Y']:.2f}" for _, row in bottom_sectors.iterrows()])

    # Preparing the string input for top and bottom companies
    top_companies_str = "\n".join([f"{row['name']} - Performance: {row['1Y']:.2f}" for _, row in top_companies.iterrows()])
    bottom_companies_str = "\n".join([f"{row['name']} - Performance: {row['1Y']:.2f}" for _, row in bottom_companies.iterrows()])

    # Combining all the data into a single string
    insights_text = (
        "Here is an overview of the top and bottom performing sectors and companies based on stock performance:\n\n"
        "📈 **Top Performing Sectors:**\n"
        + top_sectors_str +
        "\n\n📉 **Bottom Performing Sectors:**\n"
        + bottom_sectors_str +
        "\n\n🏆 **Top Performing Companies:**\n"
        + top_companies_str +
        "\n\n📉 **Bottom Performing Companies:**\n"
        + bottom_companies_str +
        "\n\nPlease analyze these results and provide insights on:\n"
        "- Key trends driving the top-performing sectors and companies for the year\n"
        "- Potential reasons behind the worst-performing sectors and companies for the year\n"
        "- Investment opportunities or risks based on these trends"
    )
    
    return insights_text

def send_insights_to_openai():
    insights_text = prepare_insights_for_openai()
    
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "user", "content": insights_text}
            ],
            max_tokens=500,  # token limit
            temperature=0.5  # temperature to control creativity of the response
        )
        
        # Store the OpenAI response
        insights = response.choices[0].message.content
        
        # Save insights to a file
        with open("openai_insights.txt", "w", encoding="utf-8") as file:
            file.write(insights)
        
        print("\n🔍 **Insights from OpenAI:**\n")
        print(insights)
        print("\nInsights saved to 'openai_insights.txt'")
    
    except Exception as e:
        print(f"Error in sending data to OpenAI: {e}")

# function to send insights and save
send_insights_to_openai()



🔍 **Insights from OpenAI:**

**Key trends driving the top-performing sectors and companies for the year:**

1. **Entertainment Sector:** The entertainment sector, led by companies like Netflix, has seen a surge in demand due to the shift towards digital streaming services during the pandemic. With people spending more time at home, the demand for entertainment content has increased, driving the performance of companies in this sector.

2. **Travel and Tourism Sector:** As travel restrictions ease and vaccination rates increase, the travel and tourism sector has started to rebound. Companies like Booking Holdings, which operate online travel platforms, have benefited from the pent-up demand for travel and vacations.

3. **Energy Services Sector:** The energy services sector, represented by companies like Baker Hughes, has seen a boost in performance as oil prices have recovered from the lows seen during the height of the pandemic. Increased demand for energy services and products as ec